In [ ]:
from bs4 import BeautifulSoup
import csv
import requests
from time import sleep
import openpyxl
import random

base_url = "https://www.olx.ba/"
add_url = "/pretraga?vrsta=samoprodaja&kategorija=23&sort_order=desc&kanton=9&grad%5B0%5D=3812&grad%5B1%5D=3969&grad%5B2%5D=5896&grad%5B3%5D=4048"
links = []


# first run(getting all apartments link)

while add_url:
    response = requests.get(f"{base_url}{add_url}", headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.text, "html.parser")
    apartments = soup.find_all("div", class_="naslov")

    for apartment in apartments:
        link = apartment.find("a", href = True)
        if link != None:
            links.append(link['href'])
            
    print(f"Finished {add_url}")
    

    try:
        next_page = soup.find("a", rel = "next")["href"]
        add_url = next_page
    
    except:
        print("Passed through all pages :)")
        break

    sleep(random.randint(2, 5))

    
    
# second run(scrape every apartment for it's info (Header, price, ...))

all_apartments = []

for link in links:
    response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.text, "html.parser")
    
    all_apartments.append({

    "header": soup.find(id = "naslovartikla").text.strip(),
    "location": soup.find(style = "font-size:14px;background-color:#4d7094;color:#fff;").text.strip(),
    "price": soup.find_all("p", style = True)[1].text.strip(),  
    "publish date": soup.find("time").text,
    "n of sqm": soup.find_all("div", class_ = "df2")[0].text,
    "floor": soup.find_all("div", class_ = "df2")[2].text,
    "n of rooms": soup.find_all("div", class_ = "df2")[1].text,  
    "furnished?": soup.find_all("div", class_ = "df2")[4].text,
    "heating system": soup.find_all("div", class_ = "df2")[3].text,  
    "link":link
        
    })
    
    print(f"Working on {link}")
    sleep(random.randint(2, 5))
    
    
# Now just write all data to csv

filename = "Apartments.csv"

f = open(filename, "w", encoding="utf-16")

headers = 'header;location;price;publish date;n of sqm;floor;n of rooms;furnished?;heating system;link \n' 

f.write(headers)

for apartment in all_apartments:
    f.write( 
            apartment.get("header") + ";" +
            apartment.get("location") + ";" +
            apartment.get("price") + ";" +
            apartment.get("publish date") + ";" +
            apartment.get("n of sqm") + ";" +
            apartment.get("floor") + ";" +
            apartment.get("n of rooms") + ";" +
            apartment.get("furnished?") + ";" +
            apartment.get("heating system") + ";" +
            apartment.get("link") + "\n"
            )
f.close()

print("You got your data :D")